In [2]:
import csv
from scipy import spatial
import numpy as np
import sys
csv.field_size_limit(sys.maxsize)

131072

In [3]:
inputFile = "pennData/pennTweetlocations.csv"
locFile = "pennData/locationlist.txt"

In [4]:
#twitter data format
#0  id
#1  message_id
#2  term
#3  category
#4  message
#5  created_time
#6  coordinates
#7  coordinates_state
#8  coordinates_address
#9  from_id
#10 in_reply_to_message_id
#11 in_reply_to_from_id
#12 retweet_message_id
#13 location
#14 friend_count
#15 followers_count
#16 time_zone
#17 lang
#18 has_hashtag
class pennTwitterData(object):
    def __init__(self, fname, flocations):
        self.fname = fname
        lines = []
        
        #TODO: BASED ON THE OLD DATA NEED TO UPDATE.
        self.fcities={}
        self.frates={}
        with open ( flocations, 'r') as f:
            for line in f.readlines():
                s_line = line.split('.')[0].split('_')
                #print(s_line)
                
                if s_line[1] in self.fcities:
                    self.fcities[s_line[1]].append(s_line[0])
                else:
                    self.fcities[ s_line[1] ] = [ s_line[0] ]
                    
                self.frates [ ",".join(s_line[:2]) ]= s_line[2]
           
    def __iter__(self):
        for line in csv.reader( open(self.fname, 'r' )):
            yield line

In [5]:
# DATA FORMAT - http://download.geonames.org/export/dump/
        #0  geonameid         : integer id of record in geonames database
        #1  name              : name of geographical point (utf8) varchar(200)
        #2  asciiname         : name of geographical point in plain ascii characters, varchar(200)
        #3  alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
        #4  latitude          : latitude in decimal degrees (wgs84)
        #5  longitude         : longitude in decimal degrees (wgs84)
        #6  feature class     : see http://www.geonames.org/export/codes.html, char(1)
        #7  feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
        #8  country code      : ISO-3166 2-letter country code, 2 characters
        #9  cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
        #10 admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
        #11 admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 
        #12 admin3 code       : code for third level administrative division, varchar(20)
        #13 admin4 code       : code for fourth level administrative division, varchar(20)
        #14 population        : bigint (8 byte int) 
        #15 elevation         : in meters, integer
        #16 dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
        #17 timezone          : the timezone id (see file timeZone.txt) varchar(40)
        #18 modification date : date of last modification in yyyy-MM-dd format
class geoLocFinder(object):
    def __init__(self, countyCodes, flocations):

        #from countycode to the County Name
        self.code2county= {}
        for line in csv.reader( open(countyCodes, 'r' ), delimiter='\t'):
            self.code2county[line[0]] = line[1:]
            
        #create nearest neighbor
        self.cord2loc = {}
        for line in csv.reader( open(flocations, 'r' ), delimiter='\t'):
            if ( line[10] == '' or line[11] == ''):
                continue
            
            countyCode = 'US.'+line[10]+'.'+line[11]
            self.cord2loc[ (float(line[4]), float(line[5]) ) ] = countyCode
            
        #nearest neighbors
        data = []
        for (lat, long) in self.cord2loc.keys():
            data.append((lat,long))
        
        self.tree = spatial.KDTree(data)
        
    def findLoc(self, latitude, longitude):
        pts = np.array([latitude, longitude])
        nn = self.tree.query(pts)
        
        loc_cords = self.tree.data[nn[1]]
        locCode = self.cord2loc[(loc_cords[0], loc_cords[1])]
        state = locCode.split('.')[1]
        if locCode in self.code2county:
            return self.code2county[locCode] + [nn[0], state]
        else:
            return [locCode, nn[0], state]

In [6]:
ptd = pennTwitterData(inputFile, locFile)

In [7]:
glf = geoLocFinder("geoname/US/us_counties.txt","geoname/US/US.txt" )

In [12]:
print( i)

18707123


In [8]:
headings = []
cs2tweets={}
badKeys = []
repeats = 0
for i, line in enumerate(ptd):
    #print(i)
    if i == 0:
        headings = line
    else:
        if ('.' in line[0]):
            repeats = repeats + 1
            continue
        #print ('continue')
        for j, item in enumerate(line):
            if headings[j] == "coordinates":
                temp = item.split("'")
                lat = float(temp[1])
                long = float(temp[3])
                loc_data = glf.findLoc(lat,long)
                if len(loc_data) < 4:
                    badKeys.append(loc_data[0])
                else:
                    state = loc_data[-1]
                    county = loc_data[0]
                    key = county + "_"+ state
                    if key in cs2tweets:
                        cs2tweets[key].append(line[4]) #message
                    else:
                        cs2tweets[key] = [ line[4] ] 
                break
        if i %100000 == 0:
            print (i)
     

In [42]:
path2output = "pennData/counties/"
for item in cs2tweets:
    f_name = "_".join(item.split())+".txt"
    with open (path2output+f_name, 'w') as f:
        f.write("\n".join(cs2tweets[item]) + "\n")

In [ ]:
glf.code2county.keys()
for key in glf.code2county.keys():
    l = key.split('.')
    #print(l[1])
    if l[1] == 'TX':
        print (key)

In [45]:
print (len(badKeys))
with open (path2output + 'badKeys.txt', 'w')as f:
    f.write( '\n'.join(list(set(badKeys))) + '\n')

In [41]:
len(cs2tweets)

2421

In [52]:
import csv
import sys
csv.field_size_limit(sys.maxsize)
# processing the data output
i = 0
heading = []
counties2rate = {}
for ll in csv.reader( open("hivRatesData/AIDSVu_County_2012.csv", 'r' )):
    if i == 0:
        heading = ll
    else:
        #
        #print (ll[3])
        county_rate = int(ll[3])
        if (county_rate <= 0):
            
        
        if (  county_rate > 0 ): 
            county_name = ll[2]
            state_name = ll[1]
            key = county_name + " " + state_name
            counties2rate[key.strip()] = [str(county_rate)]
        #else:
        #    print(county_rate)
    i = i + 1

In [57]:
len(counties2rate)

2333

In [66]:
import glob
output_path = "pennData/countiesRates/"
for i, file in enumerate(glob.glob("pennData/counties/*")):
    file_name = file.split('/')[-1]
    key_name = file_name.split('.')[0].replace("_", " ")
    if key_name != "badKeys":
        if key_name in counties2rate:
            print(i, file, key_name)
            with open ( file, 'r') as f:
                with open (output_path + "_".join(key_name.split() + counties2rate[key_name]) + ".txt", 'w') as fileOutput:
                    fileOutput.write(f.read());
#     if (name != ""):
#         key = " ".join(name [:-1])
#         state = name[-1]
#         if key.strip() != '':
#             #print (i, key)
#             if (key in counties2rate):
                
#                 if (state != counties2rate[key][0]):
#                     print (key,state, counties2rate[key])
                #with open ( file, 'r') as f:
                #    with open (output_path + "_".join(key.split() + counties2rate[key]) + ".txt", 'w') as fileOutput:
                #            fileOutput.write(f.read());

0 pennData/counties/Ottawa_County_OH.txt Ottawa County OH
1 pennData/counties/Avery_County_NC.txt Avery County NC
2 pennData/counties/Fairfield_County_SC.txt Fairfield County SC
3 pennData/counties/Onondaga_County_NY.txt Onondaga County NY
4 pennData/counties/Rabun_County_GA.txt Rabun County GA
5 pennData/counties/LaGrange_County_IN.txt LaGrange County IN
6 pennData/counties/Tippecanoe_County_IN.txt Tippecanoe County IN
7 pennData/counties/Prairie_County_AR.txt Prairie County AR
9 pennData/counties/Morrow_County_OH.txt Morrow County OH
10 pennData/counties/Centre_County_PA.txt Centre County PA
11 pennData/counties/Allegan_County_MI.txt Allegan County MI
13 pennData/counties/Glenn_County_CA.txt Glenn County CA
14 pennData/counties/Muscatine_County_IA.txt Muscatine County IA
15 pennData/counties/Berkshire_County_MA.txt Berkshire County MA
16 pennData/counties/Meigs_County_OH.txt Meigs County OH
17 pennData/counties/Ulster_County_NY.txt Ulster County NY
19 pennData/counties/Monroe_County_

In [59]:
print(i)

2421


In [5]:
print ( "\n".join(heading))

County FIPS Code
State
County Name 
County Rate 
County Rate Decile
County Rate Stability
County Cases
County Cases Decile
Male Rate
Male Rate Decile
Male Rate Stability
Male Cases
Male Cases Decile
Female Rate
Female Rate Decile
Female Rate Stability
Female Cases
Female Cases Decile
Black Rate
Black Rate Decile
Black Rate Stability
Black Cases
Black Cases Decile
White Rate
White Rate Decile
White Rate Stability
White Cases
White Cases Decile
Hispanic Rate
Hispanic Rate Decile
Hispanic Rate Stability
Hispanic Cases
Hispanic Cases Decile
Age 13-24 Rate
Age 13-24 Rate Decile
Age 13-24 Rate Stability
Age 13-24 Cases
Age 13-24 Cases Decile
Age 25-34 Rate
Age 25-34 Rate Decile
Age 25-34 Rate Stability
Age 25-34 Cases
Age 25-34 Cases Decile
Age 35-44 Rate
Age 35-44 Rate Decile
Age 35-44 Rate Stability
Age 35-44 Cases
Age 35-44 Cases Decile
Age 45-54 Rate
Age 45-54 Rate Decile
Age 45-54 Rate Stability
Age 45-54 Cases
Age 45-54 Cases Decile
Age 55+ Rate
Age 55+ Rate Decile
Age 55+ Rate Stabili

In [13]:
lines = []
with open ('pennData/temp.txt', 'r') as f:
    for line in f.readlines():
        lines.append( int(line.split()[0]))

In [23]:
s_lines = sorted(lines)
sum(lines)

5338673

In [29]:
lessthan10 = []
for i in s_lines:
    if i < 25:
        lessthan10.append(i)

In [30]:
len(lessthan10)

420

In [24]:
from statistics import *
mean(lines)

2677.368605817452

In [26]:
stdev(lines)


30088.86298038063